# Map-Reduce en Hadoop

Dans cette séance, vous allez implémenter des Map-Reduce en Hadoop.

Pour cela, le principe de fonctionnement est simple :

* un premier script Python (dit _mapper_) va lire l'entrée standard (`sys.stdin` en Python) ligne par ligne et, à chaque ligne lue, afficher le résultat de l'étape de _map_ (sur la sortie standard, via un `print()`);
* ensuite Hadoop va centraliser toutes les paires clés-valeurs émises et les trier par ordre (alphabétique) croissant ;
* enfin, un second script Python (dit _mapper_) va lire l'entrée standard (`sys.stdin` en Python) ligne par ligne (chaque ligne correspondant donc à une paire clé-valeur émise par le _mapper_, les lignes étant triées entre elles) et, pour chaque clé, afficher sur la sortie standard le résultat de l'opération de réduction.

**Attention:** pour vous assurer que tous les programmes Python que vous écrirez dans ce TD puissent s'exécuter correctement, vous vous assurerez que :

1. ils contiennent bien l'en-tête `#!/usr/bin/env python` en première ligne ;
2. ils disposent bien des droits d'exécution pour l'utilisateur courant.

**NOTE CORRIGÉ :** les fichiers Python de mappers et de reducers sont disponibles dans l'archive <https://raw.githubusercontent.com/rtavenar/m2-big_data/main/notebooks/hadoop_mapreduce/hadoop_mapreduce_corr.zip>

## 1. Préambule : remis en route de `hadoop`

Exécutez les cellules ci-dessous pour obtenir un environnement Hadoop fonctionnel.

In [1]:
# Télécharger et extraire hadoop

!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz

--2023-10-23 09:59:09--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  27.3MB/s    in 26s     

2023-10-23 09:59:35 (26.8 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]

/usr/lib/jvm/java-11-openjdk-amd64


In [22]:
import os

os.environ["JAVA_HOME"] = os.popen("dirname $(dirname $(readlink -f $(which java)))").read().strip()
os.environ["PATH"] += ":/content/hadoop-3.3.6/bin/"

**Question 1.1.** Créez un dossier `/hdfs/input` sur le HDFS.

In [6]:
!hadoop fs -mkdir -p /hdfs/input

**Question 1.2.** Téléchargez les données contenues dans l'archive <https://raw.githubusercontent.com/rtavenar/m2-big_data/main/notebooks/hadoop_mapreduce/hadoop_mapreduce_data.zip>, décompressez-les dans le dossier courant, puis copiez-les vers le dossier `/hdfs/input` du HDFS.

In [ ]:
!wget https://raw.githubusercontent.com/rtavenar/m2-big_data/main/notebooks/hadoop_mapreduce/hadoop_mapreduce_data.zip
!unzip hadoop_mapreduce_data.zip

In [53]:
!hadoop fs -copyFromLocal *.txt *.csv /hdfs/input

copyFromLocal: `/hdfs/input/CRND0103-2020-SD_Buffalo_13_ESE.txt': File exists
copyFromLocal: `/hdfs/input/CRND0103-2021-SD_Buffalo_13_ESE.txt': File exists
copyFromLocal: `/hdfs/input/CRND0103-2022-SD_Buffalo_13_ESE.txt': File exists
copyFromLocal: `/hdfs/input/lorem_0.txt': File exists
copyFromLocal: `/hdfs/input/lorem_1.txt': File exists
copyFromLocal: `/hdfs/input/lorem_2.txt': File exists
copyFromLocal: `/hdfs/input/lorem_3.txt': File exists
copyFromLocal: `/hdfs/input/lorem_4.txt': File exists


## 2. Premier exercice : calcul de maxima de température

Le but de cette partie est d'utiliser MapReduce sur Hadoop pour obtenir la valeur maximale de température **par mois** sur les trois années 2020, 2021 et 2022 dans la ville de Buffalo.

Les données correspondantes sont disponibles dans les fichiers `CRND0103-*.txt` extraits précédemment.

**Question 2.1.** Ecrire un programme `mapper_Buffalo.py` permettant de découper chaque ligne récupérée sur l'entrée standard `stdin` (lignes des fichiers de données représentant des données météo journalières), et d'en extraire le mois et la température max journalière. La date est fournie en colonne 2 des fichiers et la température max en colonne 6.

Le mois servira de clé intermédiaire et la température de valeur intermédiaire de sortie du `mapper`. Ces deux éléments seront imprimés sur la sortie standard séparés par une tabulation grâce à la fonction `print()`.

L'entrée standard `stdin` pourra être récupérée grâce au module `sys`de Python.

**Question 2.2.** Pour tester votre programme `mapper_Buffalo.py`, écrire une commande Unix permettant d'afficher le contenu des fichiers de la forme `CRND0103-*.txt*.txt` (on travaille pour l'instant sur les fichiers locaux) sur la sortie standard et de passer ce contenu sur l'entrée standard du programme `mapper_Buffalo.py`.

In [30]:
!chmod u+x *.py

In [37]:
!cat CRND0103-*.txt | ./mapper_Buffalo.py

01	8.6
01	5.8
01	2.4
01	9.7
01	4.2
01	4.6
01	0.5
01	-2.6
01	-1.7
01	-8.6
01	-9.2
01	4.5
01	-7.2
01	-14.1
01	-12.7
01	-7.2
01	0.1
01	-10.4
01	-6.3
01	-1.8
01	11.2
01	6.8
01	2.4
01	0.7
01	9.8
01	1.3
01	2.7
01	6.7
01	7.8
01	2.1
01	6.5
02	14.7
02	6.2
02	-2.5
02	-0.2
02	1.2
02	0.9
02	-3.3
02	-0.2
02	-2.7
02	-2.3
02	2.0
02	0.7
02	-7.7
02	6.0
02	3.4
02	5.9
02	0.0
02	-4.2
02	-5.6
02	4.4
02	10.8
02	10.3
02	6.4
02	1.0
02	-1.8
02	1.5
02	6.1
02	10.2
02	14.8
03	1.4
03	6.9
03	7.5
03	15.0
03	7.2
03	18.0
03	22.0
03	10.3
03	8.5
03	10.2
03	13.9
03	4.4
03	2.2
03	-2.7
03	2.3
03	-1.5
03	8.1
03	5.5
03	-0.2
03	-1.3
03	12.8
03	8.4
03	15.6
03	13.9
03	1.3
03	6.3
03	11.6
03	13.0
03	15.3
03	18.9
03	17.3
04	3.0
04	-9.0
04	-4.5
04	10.1
04	13.2
04	20.2
04	19.0
04	7.8
04	4.8
04	16.2
04	5.3
04	-3.5
04	-4.3
04	2.3
04	4.3
04	4.3
04	13.3
04	11.9
04	10.8
04	16.6
04	22.3
04	22.2
04	14.8
04	16.0
04	16.3
04	21.1
04	23.8
04	16.9
04	20.5
04	27.3
05	18.5
05	20.5
05	23.3
05	15.5
05	15.8
05	17.3
05	10.0
05	16.0
05	7.5
05	6.8
05	8

**Question 2.3.** Ecrire un programme `reducer_Buffalo.py` qui récupère les clés (mois) et valeurs (température max journalière) intermédiaires (issues des processus _mapper_) sur l'entrée standard `stdin` et qui calcule le maximum de température par mois. Le mois et la température max par mois seront imprimés sur la sortie standard comme clé et valeur de sortie du processus MapReduce.

*Indication : utiliser le fait que le `reducer` reçoive les paires de clé et valeur intermédiaires triés par clé.*

**Question 2.4.** Pour tester votre programme `reducer_Buffalo.py`, écrire une commande Unix qui permet d'enchainer les traitements du `mapper` et du `reducer`.

*Indication : on pourra utiliser la commande [`sort`](https://www.gnu.org/software/coreutils/manual/html_node/sort-invocation.html) de Unix pour simuler le traitement intermédiaire de tri effectué par Hadoop.*

In [36]:
!cat CRND0103-*.txt | ./mapper_Buffalo.py | sort | ./reducer_Buffalo.py

01	12.7
02	15.5
03	22.0
04	29.5
05	29.8
06	36.8
07	42.7
08	40.8
09	36.4
10	32.7
11	25.9
12	20.9


**Question 2.5.** Utiliser `mapred streaming` pour enchaîner les opérations, cette fois-ci en utilisant Hadoop et les fichiers localisés sur le HDFS en entrée. Vous écrirez les sorties de votre traitement dans `/hdfs/output_Buffalo`

In [38]:
!mapred streaming -file mapper_Buffalo.py -mapper mapper_Buffalo.py -file reducer_Buffalo.py  -reducer reducer_Buffalo.py -input /hdfs/input/CRND0103-*.txt -output /hdfs/output_Buffalo

2023-10-23 14:29:30,103 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper_Buffalo.py, reducer_Buffalo.py] [] /tmp/streamjob2083452419361744149.jar tmpDir=null
2023-10-23 14:29:30,980 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-10-23 14:29:31,139 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-10-23 14:29:31,139 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2023-10-23 14:29:31,157 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2023-10-23 14:29:31,360 INFO mapred.FileInputFormat: Total input files to process : 3
2023-10-23 14:29:31,386 INFO mapreduce.JobSubmitter: number of splits:3
2023-10-23 14:29:31,634 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1634697882_0001
2023-10-23 14:29:31,634 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-10-23 14:29:32,044 INFO mapred.Local

**Question 2.6.** Comment s'appelle le fichier dans lesquel a été écrit le résultat ? Affichez son contenu.

In [39]:
!hadoop fs -ls /hdfs/output_Buffalo

Found 2 items
-rw-r--r--   1 root root          0 2023-10-23 14:29 /hdfs/output_Buffalo/_SUCCESS
-rw-r--r--   1 root root         96 2023-10-23 14:29 /hdfs/output_Buffalo/part-00000


In [40]:
!hadoop fs -cat /hdfs/output_Buffalo/part-00000

01	12.7
02	15.5
03	22.0
04	29.5
05	29.8
06	36.8
07	42.7
08	40.8
09	36.4
10	32.7
11	25.9
12	20.9


## 3. Deuxième exercice : _wordcount_

Dans cette partie, vous devrez implémenter un wordcount **en Hadoop MapReduce**.

Pour cela vous devrez :

* travailler sur les fichiers `lorem_*.txt` ;
* implémenter un `mapper_wordcount.py` qui affichera, pour chaque mot lu, une sortie du type `le_mot_lu 1` ;
* implémenter un `reducer_wordcount.py` qui fera la somme des comptes mot par mot ;
* afficher le résultat pour vous assurer que tout a bien fonctionné.

In [24]:
!chmod u+x *.py

In [26]:
!cat lorem_*.txt | ./mapper_wordcount.py

Commodi 1
ratione 1
explicabo 1
saepe 1
accusantium 1
voluptas 1
illum 1
rem 1
assumenda 1
rerum 1
delectus 1
saepe 1
sit 1
doloribus 1
quos 1
nulla 1
praesentium 1
voluptatibus 1
accusantium 1
Perspiciatis 1
voluptatibus 1
optio 1
maiores 1
dignissimos 1
veniam 1
at 1
magni 1
quo 1
similique 1
similique 1
omnis 1
quo 1
enim 1
aperiam 1
quae 1
ipsum 1
earum 1
cumque 1
inventore 1
natus 1
omnis 1
beatae 1
rerum 1
similique 1
porro 1
deserunt 1
exercitationem 1
rem 1
hic?Consequatur 1
blanditiis 1
reiciendis 1
rem 1
repellat 1
dolore 1
quod 1
vel 1
nam 1
placeat 1
cumque 1
cum 1
illum 1
voluptate 1
Soluta 1
alias 1
eum 1
illum 1
dolor 1
cupiditate 1
dolores 1
praesentium 1
neque 1
aperiam 1
incidunt 1
adipisci 1
maiores 1
earum 1
quia 1
rem 1
natus 1
necessitatibus 1
nihil 1
labore 1
reprehenderit?Quia 1
aliquid 1
nostrum 1
ab 1
eum 1
excepturi 1
architecto 1
autem 1
incidunt 1
recusandae 1
facere 1
voluptates 1
architecto 1
temporibus 1
animi 1
provident 1
error 1
eos 1
vitae 1
facilis 

In [41]:
!cat lorem_*.txt | ./mapper_wordcount.py | sort | ./reducer_wordcount.py

a	2
ab	4
accusamus	1
accusantium	2
ad	1
adipisci	1
alias	3
aliquam	2
aliquid	2
amet	3
animi	3
aperiam	4
Aperiam	1
architecto	2
asperiores	3
assumenda	2
Assumenda	1
at	3
atque	1
aut	1
autem	3
beatae	4
blanditiis	4
Commodi	1
consectetur	2
consequuntur	1
corporis	3
corrupti	2
culpa	3
cum	1
cumque	4
cupiditate	2
delectus	3
Delectus	1
deleniti	2
deserunt	3
dicta	1
dignissimos	4
distinctio	1
dolor	5
Dolor	1
dolore	3
doloremque	2
dolores	2
doloribus	2
dolorum	1
ducimus	2
ea	5
eaque	1
earum	5
eius	1
eligendi	2
enim	2
eos	4
error	3
esse	1
est	5
eum	3
eveniet	1
ex	2
excepturi	3
exercitationem	2
expedita	2
explicabo	4
facere	3
facilis	1
fuga	2
fugiat	1
fugit	1
harumTempora	1
hic	1
hicBeatae	1
hic?Consequatur	1
id	3
illum	4
impedit	2
in	1
incidunt	4
inventore	4
ipsam	1
ipsum	2
iste	1
itaque	1
Iusto	1
labore	1
laboriosam	3
Laboriosam	1
laborum	1
laudantium	2
libero	4
magnam	1
magni	3
maiores	4
minima	3
minus	3
modi	1
molestiae	3
molestiae?	1
mollitia	6
nam	4
natus	5
necessitatibus	3
nemo	1
neque	1


In [42]:
!mapred streaming -file mapper_wordcount.py -mapper mapper_wordcount.py -file reducer_wordcount.py  -reducer reducer_wordcount.py -input /hdfs/input/lorem_*.txt -output /hdfs/output_wordcount

2023-10-23 14:31:15,748 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper_wordcount.py, reducer_wordcount.py] [] /tmp/streamjob14994637074962929923.jar tmpDir=null
2023-10-23 14:31:16,339 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-10-23 14:31:16,512 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-10-23 14:31:16,513 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2023-10-23 14:31:16,529 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2023-10-23 14:31:16,697 INFO mapred.FileInputFormat: Total input files to process : 5
2023-10-23 14:31:16,720 INFO mapreduce.JobSubmitter: number of splits:5
2023-10-23 14:31:16,978 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local288181184_0001
2023-10-23 14:31:16,979 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-10-23 14:31:17,344 INFO mapred.L

In [43]:
!hadoop fs -cat /hdfs/output_wordcount/part-00000

Aperiam	1
Assumenda	1
Beatae	1
Commodi	1
Consequatur	1
Delectus	1
Dolor	1
Iusto	1
Laboriosam	1
Nesciunt	1
Perspiciatis	1
Possimus	1
Quia	1
Repellat	1
Soluta	1
Tempora	1
a	2
ab	4
accusamus	1
accusantium	2
ad	1
adipisci	1
alias	3
aliquam	2
aliquid	2
amet	3
animi	3
aperiam	4
architecto	2
asperiores	3
assumenda	2
at	3
atque	1
aut	1
autem	3
beatae	4
blanditiis	4
consectetur	2
consequuntur	1
corporis	3
corrupti	2
culpa	3
cum	1
cumque	4
cupiditate	2
delectus	3
deleniti	2
deserunt	3
dicta	1
dignissimos	4
distinctio	1
dolor	5
dolore	3
doloremque	2
dolores	2
doloribus	2
dolorum	1
ducimus	2
ea	5
eaque	1
earum	5
eius	1
eligendi	2
enim	2
eos	4
error	3
esse	1
est	5
eum	3
eveniet	1
ex	2
excepturi	3
exercitationem	2
expedita	2
explicabo	4
facere	3
facilis	1
fuga	2
fugiat	1
fugit	1
harum	1
hic	2
hic?	1
id	3
illum	4
impedit	2
in	1
incidunt	4
inventore	4
ipsam	1
ipsum	2
iste	1
itaque	1
labore	1
laboriosam	3
laborum	1
laudantium	2
libero	4
magnam	1
magni	3
maiores	4
minima	3
minus	3
modi	1
molestiae	3
mol

**Question subsidiaire.** Améliorer votre code (si besoin) pour que la ponctuation soit correctement prise en compte.

## Pour aller plus loin

Faites de même en utilisant les fichiers `notes_*.csv` pour calculer les moyennes par étudiant (identifié par son numéro d'étudiant, première colonne).
Vous considérerez que les fichiers `notes_*.csv` contiennent une note par ligne et que les moyennes sont à calculer sans tenir compte de quelconques coefficients et en mélangeant toutes les matières.

In [47]:
!chmod u+x *.py

In [49]:
!cat notes_*.csv | ./mapper_notes.py

21949510	STATS	10
21881510	BIG-DATA	19
22001510	ML	16
22001510	STATS	6
22001510	STATS	11
21881510	STATS	10
21881510	STATS	20
21881510	BIG-DATA	19
21991510	BIG-DATA	19
21949510	BIG-DATA	6
21881510	BIG-DATA	10
22001510	BIG-DATA	6
22001510	ML	16
21881510	ML	18
21991510	BIG-DATA	12
21949510	STATS	12
21881510	ML	7
21881510	ML	12
21881510	STATS	16
21881510	ML	15
21991510	ML	14
21881510	BIG-DATA	9
21949510	ML	18
21881510	ML	16
21949510	ML	18
21949510	BIG-DATA	8
21991510	ML	6
21949510	STATS	5
22001510	ML	15
21881510	BIG-DATA	20
21881510	BIG-DATA	18
22001510	STATS	18
22001510	ML	14
21991510	BIG-DATA	9
21881510	STATS	16
21991510	ML	6
21949510	STATS	11
22001510	STATS	9
21881510	STATS	14
21881510	BIG-DATA	12
21949510	BIG-DATA	5
22001510	STATS	18
22001510	BIG-DATA	10
22001510	ML	19
21949510	ML	9
21881510	STATS	17
21949510	STATS	20
21881510	STATS	14
21881510	STATS	8
21949510	ML	14
21881510	ML	7
21949510	ML	17
21949510	STATS	13
21949510	ML	12
21949510	ML	13
21949510	ML	20
22001510	STATS	17
21881510	M

In [61]:
!cat notes_*.csv | ./mapper_notes.py | sort | ./reducer_notes.py

21881510	13.064516129032258
21949510	12.633333333333333
21991510	11.235294117647058
22001510	12.772727272727273


In [62]:
!mapred streaming -file mapper_notes.py -mapper mapper_notes.py -file reducer_notes.py  -reducer reducer_notes.py -input /hdfs/input/notes_*.csv -output /hdfs/output_notes3

2023-10-23 15:02:10,365 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper_notes.py, reducer_notes.py] [] /tmp/streamjob2438078788322334696.jar tmpDir=null
2023-10-23 15:02:10,920 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-10-23 15:02:11,037 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-10-23 15:02:11,037 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2023-10-23 15:02:11,053 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2023-10-23 15:02:11,235 INFO mapred.FileInputFormat: Total input files to process : 5
2023-10-23 15:02:11,258 INFO mapreduce.JobSubmitter: number of splits:5
2023-10-23 15:02:11,505 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local354613490_0001
2023-10-23 15:02:11,505 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-10-23 15:02:11,917 INFO mapred.LocalDistr

In [63]:
!hadoop fs -cat /hdfs/output_notes3/part-00000

21881510	13.064516129032258
21949510	12.633333333333333
21991510	11.235294117647058
22001510	12.772727272727273
